# Introduction

 In this project, we intend to help a client to open Mexican resturant chain in one of the most multicultural cities in Manhattan, New York, USA or Toronto, Canada. However, the client wants to be well informed where it would be more profitable and safe to invest. The task upon us is to analyse the venues/locations in these two different cities, that provide favourable conditions for the resturant to succeed.
 
 As the part of this, we will analyse the Buisness venues in the neighborhood's of both cities. We will use frequency of Resturants and Diners in different neighborhoods, considering only non native resturants. Since, the client wants to open a Mexican resturant, It will be useful to study the acceptance of foreign cuisine  by the natives at both the cities.

## Data

We will gather all the resturant data using the County/Neighborhood information for both cities using the below links and using them along with their location co-ordinates with Foursquare to get the information on venues and their popularity among the citizens of the area. We will further filter data the exculdes central europe and North american cuisine.

Toronto: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

New York:https://cocl.us/new_york_dataset

The most important data columns needed for the analysis are the following:

1. Neighborhood/County
2. Neighborhood/County Latitude
3. Neighborhood/County Longitude
4. Venue
5. Venue Category
6. Venue Latitude
7. Veneue Longitude

# Pacakges

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


# New York

In [3]:
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('new_york_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan


# Toronto

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)
list_toronto = data[0]
list_toronto = list_toronto[list_toronto['Borough'] != 'Not assigned']
location_data = pd.read_csv("Geospatial_Coordinates.csv")
list_toronto_latlog = pd.merge(list_toronto, location_data, on='Postal Code')
list_toronto_latlog = list_toronto_latlog[list_toronto_latlog["Borough"].apply(lambda x: "Toronto" in x)]

In [7]:
latitude = 43.651070
longitudnal = -79.347015

# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitudnal], zoom_start=11)

# add markers to map
for lat, lng, label in zip(list_toronto_latlog['Latitude'], list_toronto_latlog['Longitude'], list_toronto_latlog['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto